In [164]:
import boto3
from boto.s3.connection import S3Connection
import boto
from boto.s3.key import Key
import sys
import os
import json
import pandas as pd
from collections import defaultdict
from matplotlib import pyplot as plt
import ast 

# %matplotlib inline
sys.path.insert(0,os.path.abspath('..'))

from utils.credentials import access_key, secret_access_key

In [4]:
client = boto3.client('s3')

conn = boto.s3.connect_to_region('us-east-2',
       aws_access_key_id=access_key,
       aws_secret_access_key=secret_access_key,
       is_secure=True,
       calling_format = boto.s3.connection.OrdinaryCallingFormat(),
       )

bucket = conn.get_bucket('persproj')

In [5]:
%%time
review_df = pd.read_csv('s3://persproj/data/review_df.csv')

In [7]:
%%time
business_df = pd.read_csv('s3://persproj/data/business_df.csv')

CPU times: user 3.1 s, sys: 1.57 s, total: 4.67 s
Wall time: 1min 4s


In [10]:
review_df.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,ujmEBvifdJM6h6RLv4wQIg,0,2013-05-07 04:34:36,1.0,Q1sbwvVQXV2734tPgoKj4Q,1.0,Total bill for this horrible service? Over $8G...,6.0,hG7b0MtEbXx5QzbzE6C_VA
1,NZnhc2sEQy3RmzKTZnqtwQ,0,2017-01-14 21:30:33,0.0,GJXCdrto3ASJOqKeVWPi6Q,5.0,I *adore* Travis at the Hard Rock's new Kelly ...,0.0,yXQM5uF2jS6es16SJzNHfg
2,WTqjgwHlXbSFevF32_DJVw,0,2016-11-09 20:09:03,0.0,2TzJjDVDEuAW6MR5Vuc1ug,5.0,I have to say that this office really has it t...,3.0,n6-Gk65cPZL6Uz8qRm3NYw
3,ikCg8xy5JIg_NGPx-MSIDA,0,2018-01-09 20:56:38,0.0,yi0R0Ugj_xUx_Nek0-_Qig,5.0,Went in for a lunch. Steak sandwich was delici...,0.0,dacAIZ6fTM6mqwW5uxkskg
4,b1b1eb3uo-w561D0ZfCEiQ,0,2018-01-30 23:07:38,0.0,11a8sVPMUFtaC7_ABRkmtw,1.0,Today was my second out of three sessions I ha...,7.0,ssoyf2_x0EQMed6fgHeMyQ


In [16]:
print('Number of businesses:', len(review_df.business_id.unique()))

Number of businesses: 192608


In [14]:
review_df.groupby('business_id').agg({'review_id': len}).sort_values('review_id', ascending=False).head(20)

,review_id
business_id,
4JNXUYY8wbaaDmk3BPzlWw,8570
RESDUcs7fIiihp38-d6_6g,8568
K7lWdNUhCbcnEvI0NhGewg,6887
f4x1YBxkLrZg652xt2KR5g,5847
cYwJA2A6I12KNkm2rtXd5g,5575
DkYS3arLOhA8si5uUEmHOw,5206
2weQS-RnoOBhb1KsHKyoSQ,4534
5LNZ67Yw9RD6nf4_UhXOjw,4522
iCQpiavjjPzJ5_3gPD5Ebg,4351


Some buisnesses and tons of reviews

### filter out only restaurant reviews

In [134]:
food_df = (business_df[
    (business_df['categories'].str.contains('Food',na=False)) |
     (business_df['categories'].str.contains('Restaurant',na=False))|
     (business_df['categories'].str.contains('Bar',na=False))
])

In [135]:
%%time
food_review_df = food_df.merge(review_df, how='inner', left_on='business_id', right_on='business_id')

CPU times: user 58.2 s, sys: 8min 40s, total: 9min 38s
Wall time: 51min 33s


In [140]:
food_review_df.to_csv('s3://persproj/data/food_review_df.csv', index=False)

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,...,stars_x,state,cool,date,funny,review_id,stars_y,text,useful,user_id
0,30 Eglinton Avenue W,"{'RestaurantsReservations': 'True', 'GoodForMe...",QXAEGFB4oINsVuTFxEYKFQ,"Specialty Food, Restaurants, Dim Sum, Imported...",Mississauga,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...",1,43.605499,-79.652289,Emerald Chinese Restaurant,...,2.5,ON,0,2017-01-27 21:54:30,2.0,6W0MQHmasK0IsaoDo4bmkw,3.0,My girlfriend and I went for dinner at Emerald...,3.0,2K62MJ4CJ19L8Tp5pRfjfQ
1,30 Eglinton Avenue W,"{'RestaurantsReservations': 'True', 'GoodForMe...",QXAEGFB4oINsVuTFxEYKFQ,"Specialty Food, Restaurants, Dim Sum, Imported...",Mississauga,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...",1,43.605499,-79.652289,Emerald Chinese Restaurant,...,2.5,ON,0,2013-06-24 23:11:30,0.0,BeeBfUxvzD4qNX4HxrgA5g,3.0,We've always been there on a Sunday so we were...,0.0,A0kENtCCoVT3m7T35zb2Vg
2,30 Eglinton Avenue W,"{'RestaurantsReservations': 'True', 'GoodForMe...",QXAEGFB4oINsVuTFxEYKFQ,"Specialty Food, Restaurants, Dim Sum, Imported...",Mississauga,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...",1,43.605499,-79.652289,Emerald Chinese Restaurant,...,2.5,ON,0,2016-01-04 12:59:22,0.0,A1D2kUnZ0HTroFreAheNSg,3.0,"***No automatic doors, not baby friendly!*** I...",0.0,SuOLY03LW5ZcnynKhbTydA
3,30 Eglinton Avenue W,"{'RestaurantsReservations': 'True', 'GoodForMe...",QXAEGFB4oINsVuTFxEYKFQ,"Specialty Food, Restaurants, Dim Sum, Imported...",Mississauga,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...",1,43.605499,-79.652289,Emerald Chinese Restaurant,...,2.5,ON,0,2014-05-09 02:38:43,0.0,2pf45Stf-pNew-xgTababQ,1.0,"Horrible service,\nI went there tonight with m...",1.0,lymyUak6KNcNKoDbK87MiQ
4,30 Eglinton Avenue W,"{'RestaurantsReservations': 'True', 'GoodForMe...",QXAEGFB4oINsVuTFxEYKFQ,"Specialty Food, Restaurants, Dim Sum, Imported...",Mississauga,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...",1,43.605499,-79.652289,Emerald Chinese Restaurant,...,2.5,ON,2,2011-03-21 14:39:55,1.0,RHhlmL07evgAdPaXQV8Omg,4.0,One of the gauges of a good Chinese restaurant...,2.0,6vU0I5XgCv9OQHZ76rV6qw


## dim sum restaurant reviews

In [157]:
dim_sum_df = food_review_df[food_review_df['categories'].str.contains('Dim Sum')]

In [309]:
len(dim_sum_df[:1000])

1000

In [352]:
import utils.preprocessing as prep
import utils.ldamodel as lda
import imp
imp.reload(lda)

<module 'utils.ldamodel' from '/Users/miraekim/workspace/coursework/persproj/utils/ldamodel.py'>

Filter out tokens in the dictionary by their frequency.

Parameters
- no_below (int, optional) – Keep tokens which are contained in at least no_below documents.

- no_above (float, optional) – Keep tokens which are contained in no more than no_above documents (fraction of total corpus size, not an absolute number).

- keep_n (int, optional) – Keep only the first keep_n most frequent tokens.

- keep_tokens (iterable of str) – Iterable of tokens that must stay in dictionary after filtering.

*Notes*

This removes all tokens in the dictionary that are:

- Less frequent than no_below documents (absolute number, e.g. 5) or (don't appear 

- More frequent than no_above documents (fraction of the total corpus size, e.g. 0.3).

- After (1) and (2), keep only the first keep_n most frequent tokens (or keep all if keep_n=None).

After the pruning, resulting gaps in word ids are shrunk. Due to this gap shrinking, the same word may have a different word id before and after the call to this function!

In [353]:
df = dim_sum_df[:1000]

filter_params = {
    'no_below': 5, # need to be in at least <no_below> documents 
    'no_above':0.5, # need to be in less than <no_above>% of documents, the lower this is, the more words it filters oout
    'keep_n': 10000,
}

lda_class = lda.ldaModel(df,
            'text',
            filter_params)

original dict len:  3847
after filter dict len:  1054


In [357]:
model_params = {
    'num_topics': 10,
    'passes':1,
    'workers': 4,
    'chunksize': 500,
    'eta':0.1,
    'alpha':0.9,
    'id2word': lda_class.dictionary
}

model = lda_class.run_model(**model_params)

In [358]:
model.print_topics()

[(0,
  '0.035*"noodl" + 0.023*"soup" + 0.022*"place" + 0.016*"good" + 0.014*"come" + 0.014*"like" + 0.013*"chines" + 0.012*"order" + 0.011*"beef" + 0.010*"restaur"'),
 (1,
  '0.023*"noodl" + 0.023*"good" + 0.022*"soup" + 0.016*"servic" + 0.016*"chines" + 0.016*"like" + 0.016*"place" + 0.015*"dish" + 0.014*"order" + 0.012*"restaur"'),
 (2,
  '0.031*"noodl" + 0.023*"good" + 0.019*"order" + 0.018*"servic" + 0.015*"place" + 0.014*"soup" + 0.014*"great" + 0.012*"beef" + 0.011*"dish" + 0.011*"price"'),
 (3,
  '0.032*"noodl" + 0.022*"come" + 0.017*"place" + 0.017*"beef" + 0.016*"like" + 0.015*"soup" + 0.014*"good" + 0.013*"restaur" + 0.012*"order" + 0.011*"servic"'),
 (4,
  '0.027*"noodl" + 0.021*"place" + 0.019*"good" + 0.017*"soup" + 0.017*"come" + 0.016*"order" + 0.014*"servic" + 0.014*"chines" + 0.013*"like" + 0.010*"restaur"'),
 (5,
  '0.035*"noodl" + 0.022*"place" + 0.014*"servic" + 0.014*"soup" + 0.013*"come" + 0.013*"order" + 0.012*"like" + 0.012*"beef" + 0.011*"chines" + 0.011*"rice"

In [361]:
from gensim.models import CoherenceModel
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=model,
                                     texts=lda_class.processed_docs,
                                     dictionary=lda_class.dictionary,
                                     coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.42342811999801705


## Model Tuning

In [ ]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [ ]:
import tqdm

grid = {}
grid['Validation_Set'] = {}

# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [# gensim.utils.ClippedCorpus(corpus, num_of_docs*0.25), 
               # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), 
               # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.75), 
               corpus]

corpus_title = ['100% Corpus']

model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=(len(beta)*len(alpha)*len(topics_range)*len(corpus_title)))
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('lda_tuning_results.csv', index=False)
    pbar.close()

In [362]:
import pyLDAvis.gensim
import pickle 
import pyLDAvis

# Visualize the topics
pyLDAvis.enable_notebook()

LDAvis_prepared = pyLDAvis.gensim.prepare(model, lda_class.bow_corpus, lda_class.dictionary)

LDAvis_prepared

/Users/miraekim/miniconda3/envs/main_env/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8      0.001125 -0.002882       1        1  13.083199
3     -0.006040 -0.002323       2        1  11.540769
1      0.003766  0.003807       3        1  10.620909
6     -0.000005  0.004566       4        1  10.563603
5     -0.002296 -0.004105       5        1  10.400094
0     -0.005123  0.002606       6        1   9.607294
9      0.002362  0.001054       7        1   9.577060
4     -0.002262  0.001364       8        1   9.095261
2      0.005738 -0.003133       9        1   8.058390
7      0.002736 -0.000953      10        1   7.453429, topic_info=    Category         Freq      Term        Total  loglift  logprob
23   Default   715.000000      good   715.000000  30.0000  30.0000
5    Default   510.000000    chines   510.000000  29.0000  29.0000
87   Default   248.000000     great   248.000000  28.0000  28.0000
55   Default   536.000000    servic   536.000000  27.0000  27.0000
120  Default   699.000000     place   699.000000  26.0000  26.0000
331  Default   635.000000      soup   635.000000  25.0000  25.0000
38   Default  1131.000000     noodl  1131.000000  24.0000  24.0000
39   Default   561.000000     order   561.000000  23.0000  23.0000
46   Default   309.000000     price   309.000000  22.0000  22.0000
6    Default   549.000000      come   549.000000  21.0000  21.0000
22   Default   215.000000        go   215.000000  20.0000  20.0000
44   Default   213.000000      pork   213.000000  19.0000  19.0000
61   Default   193.000000      tabl   193.000000  18.0000  18.0000
93   Default   342.000000      time   342.000000  17.0000  17.0000
4    Default   221.000000   chicken   221.000000  16.0000  16.0000
376  Default   443.000000      beef   443.000000  15.0000  15.0000
16   Default   436.000000      dish   436.000000  14.0000  14.0000
980  Default   251.000000      vega   251.000000  13.0000  13.0000
165  Default   225.000000    pretti   225.000000  12.0000  12.0000
34   Default   128.000000      menu   128.000000  11.0000  11.0000
37   Default   148.000000     night   148.000000  10.0000  10.0000
220  Default   126.000000     think   126.000000   9.0000   9.0000
262  Default   148.000000      star   148.000000   8.0000   8.0000
309  Default   177.000000     asian   177.000000   7.0000   7.0000
698  Default   151.000000     spici   151.000000   6.0000   6.0000
32   Default   127.000000      meal   127.000000   5.0000   5.0000
114  Default   477.000000      like   477.000000   4.0000   4.0000
13   Default   127.000000    delici   127.000000   3.0000   3.0000
427  Default   105.000000       say   105.000000   2.0000   2.0000
52   Default   151.000000      sauc   151.000000   1.0000   1.0000
..       ...          ...       ...          ...      ...      ...
120  Topic10    56.619133     place   699.382751   0.0826  -3.9115
115  Topic10    15.699526      look   171.253586   0.2070  -5.1942
23   Topic10    55.747738      good   715.720642   0.0440  -3.9270
87   Topic10    21.430918     great   248.596954   0.1455  -4.8830
34   Topic10    11.862389      menu   128.723969   0.2122  -5.4745
262  Topic10    13.392622      star   148.402390   0.1913  -5.3531
40   Topic10     7.850611    overal    80.996742   0.2627  -5.8873
243  Topic10    14.767222      best   167.817078   0.1660  -5.2554
170  Topic10    15.187340    shrimp   174.323486   0.1560  -5.2274
16   Topic10    32.717487      dish   436.052338   0.0066  -4.4599
309  Topic10    15.314216     asian   177.386520   0.1469  -5.2191
5    Topic10    36.762592    chines   510.689178  -0.0348  -4.3434
376  Topic10    32.215080      beef   443.738373  -0.0263  -4.4754
124  Topic10    25.984463      rice   361.590851  -0.0365  -4.6903
981  Topic10    18.259176  venetian   231.291138   0.0575  -5.0432
331  Topic10    38.756821      soup   635.346680  -0.2004  -4.2905
106  Topic10    21.233284       fri   290.858551  -0.0208  -4.8923
38   Topic10    57.975513 